# First protocol
_Code written and runs in python 3.11.0. Modify environment variables and queries as needed._  
_Please use venv_

## Protocol
· Start with keywords:

- Smartchain

- Nft

- Airdrop

- Crypto

- …etc.

1. Sample up to 10k tweets containing at least one term from 100 random hours from the past year (so 1M tweets)

2. Determine the most engaged (top) with users from this combined sample (100 or 1000)

3. Pull up to 1000 comments for each top user

4. Determine top users whose comments mention at least three users other than the top user

5. Expand top user sample if we don’t have at least 100 airdrop seeders

6. Time series chart plots:

7. Top user activity

8. Airdrop seeder activity

9. Negative reaction activity? (based on sentiment analysis of replies to airdrop messages)

10. External crypto value signals (from where?)



# Dependencies
Run the following commands in the terminal to install the required packages

$pip install requests  
  
$pip install pandas  
  
$pip install datetime  
  
$pip install python-dateutil
  

--------------------  
create a files <data.json> in the same folder as this jupyter notebook


# Authentication step
In the code cell below replace bearer_token with your bearer token. Run the cell, then delete your bearer token.
This creates the token as an environment variable to be used under the name TOKEN. The token can then be removed so that others do not have access to your token when code is shared via GitHub. I will change this to dotenv and a .gitignore file later I just havent done that yet.

In [64]:
os.environ['TOKEN'] = ''

In [84]:
"""This cell creates functions to be used for authentication as well as creating endpoints."""
import requests

def auth():
    """Retrieves your bearer token."""
    return os.getenv('TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def create_url(keyword, start_date, end_date, max_results = 100):
    
    #Change to the endpoint you want to collect data from
    search_url = "https://api.twitter.com/2/tweets/search/all" 

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id',
                    'tweet.fields': 'id,created_at,text,author_id,public_metrics',
                    'user.fields': 'id,name,username,created_at',
                    #'place.fields': 'country',
                    'next_token': {}}
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

#print(auth())

In [4]:
import datetime
import random
"""This code cell contains two functions (is_leap_year and random_date) which help generate a random one hour date range when random_date() is called"""
# Use by calling "start_time, end_time = random_date()"

def is_leap_year(year):
    """Returns True if the given year is a leap year, False otherwise."""
    if year % 4 == 0:
        if year % 100 == 0:
            if year % 400 == 0:
                return True
            else:
                return False
        else:
            return True
    else:
        return False

def random_date():
    """Generate a random one hour date range within the last year in RFC 3339 format to be used with twitter API."""
    year = random.randint(datetime.datetime.now().year - 1, datetime.datetime.now().year)
    month = random.randint(1, datetime.datetime.now().month)
    if month == datetime.datetime.now().month:
        day = random.randint(1, datetime.datetime.now().day - 1)
    elif month == 2:
        if is_leap_year(year):
            day = random.randint(1, 29)
        else:
            day = random.randint(1, 28)
    elif month in [1, 3, 5, 7, 8, 10, 12]:
        day = random.randint(1, 31)
    else:
        day = random.randint(1, 31)
    hour = random.randint(0, 23)
    start_time = datetime.datetime(year, month, day, hour)
    end_time = start_time + datetime.timedelta(hours=1)
    start_timestamp = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_timestamp = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    return start_timestamp, end_timestamp

#start_time1, end_time1 = random_date()
#print(f"The one hour range is from {start_time1} to {end_time1}")

In [ ]:
"""WORKING EXAMPLE OF SO FAR, Above is for more function use etc."""
import json
import pandas as pd
import csv
from typing import Optional
import time

def tweets_per_range(keyword: str, start_times_list: list, end_times_list: list, results_per_range: int, next_token: Optional[str] = None) -> json:
    bearer_token = auth()
    headers = create_headers(bearer_token)
    json_data = []
    for i in range(0, len(start_times_list)):
        total_count = 0
        max_results = 100
        url = create_url(keyword, start_times_list[i], end_times_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        json_data.append(json_response["data"])
        total_count += json_response['meta']['result_count']
        #if (total_count <= results_per_range) and (if 'next_token' in json_response['meta']):
            #next_token_st = json_response['meta']['next_token']
            #tweets_per_range(keyword: , start_times_list[i], end__times_list[i], results_per_range, next_token_st)

    while total_count <= results_per_range:
        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
            url = create_url(keyword, start_times_list[i], end_times_list[i], max_results)
            json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
            json_data.append(json_response['data'])
            total_count += json_response['meta']['result_count']
            time.sleep(5)
        else:
            print("Max results reached before desired amount of tweets.")
            break
    return(json_data)




# Main Working Cell

The below cell is intended to be the final cell which ties together all functions into the 'solution' to the protocol

In [ ]:
# Use the commented out code below when using full access api
# #start_time, end_time = random_date()
start_list = ["2023-01-08T17:00:00Z", "2023-01-10T17:00:00Z", "2023-01-11T17:00:00Z"]
end_list = ["2023-01-08T18:00:00Z", "2023-01-10T18:00:00Z", "2023-01-11T18:00:00Z"]

json_final_data = tweets_per_range("Smartchain OR Airdrop OR Crypto OR Nft", start_list, end_list, 200)

json_to_file = json.dumps(json_final_data, indent=4)
with open("data.json", "w") as outfile:
    outfile.write(json_to_file)